In [ ]:
# import necessary packages for hand gesture recognition project using Python OpenCV
import os
import time
import subprocess
import cv2
import vlc
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model

2023-01-06 20:21:38.977510: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def capture_frames(webcam):
    # Read each frame from the webcam
    _, frame = webcam.read()

    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    # Show the final output
    cv2.imshow("Output", frame)
    return frame
    

In [ ]:
def get_hand_landmarks(hands, frame):
    x , y, c = frame.shape
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Get hand landmark prediction
    result = hands.process(framergb)
    
    # import pdb; pdb.set_trace()
    if not result.multi_hand_landmarks:
        # no hands detected
        return []
    
    # post process the result
    landmarks = []
    for handslms in result.multi_hand_landmarks:
        for lm in handslms.landmark:
            # print(id, lm)
            lmx = int(lm.x * x)
            lmy = int(lm.y * y)
            landmarks.append([lmx, lmy])
    return landmarks

In [ ]:
def identify_gesture(model, landmarks, classNames):
    # Predict gesture
    prediction = model.predict([landmarks])
    # print(prediction)
    classID = np.argmax(prediction)
    className = classNames[classID]
    return className
    

In [ ]:
def invoke_music_controls(media_player, class_name):
    
    print(f"{class_name} invoked")
    if class_name == "thumbs up":
        
        media_player.play()
    
    if class_name == "stop":
        media_player.stop()
    

In [ ]:
def get_media_player(player_name="vlc"):
    player = vlc.MediaPlayer("StarWars60.wav")
    return player

In [ ]:
def run_loop(media_player, webcam, hands, model, class_names, delay=0.5):
    while True:
        class_name = None
        landmarks = None
        # Read each frame from the webcam
        frames = capture_frames(webcam)
        landmarks = get_hand_landmarks(hands, frames)
        if landmarks:
            class_name = identify_gesture(model, landmarks, class_names)
            
        if class_name:
            invoke_music_controls(media_player, class_name)

        if cv2.waitKey(1) == ord('q'):
                break

        time.sleep(delay)

In [ ]:
def monitor_for_hand_gesture():
    # initialize mediapipe
    mpHands = mp.solutions.hands
    hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    mpDraw = mp.solutions.drawing_utils
    
    # Load the gesture recognizer model
    model = load_model('hand-gesture-recognition-code/mp_hand_gesture')

    # Load class names
    f = open('hand-gesture-recognition-code/gesture.names', 'r')
    class_names = f.read().split('\n')
    f.close()
    # print(classNames)
    
    # Initialize the webcam
    webcam = cv2.VideoCapture(0)
    
    # get media player
    media_player = get_media_player()

    try:
        run_loop(media_player, webcam, hands, model, class_names)
    except Exception as e:
        raise
    finally:
        # release the webcam and destroy all active windows
        webcam.release()
        webcam.destroyAllWindows()
        #cv2.waitKey(1) 


In [ ]:
monitor_for_hand_gesture()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2023-01-06 20:21:46.541547: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 0s 142ms/step
okay invoked
1/1 [==============================] - 0s 38ms/step
okay invoked
1/1 [==============================] - 0s 37ms/step
okay invoked
1/1 [==============================] - 0s 52ms/step
okay invoked
1/1 [==============================] - 0s 39ms/step
rock invoked
1/1 [==============================] - 0s 44ms/step
rock invoked
1/1 [==============================] - 0s 38ms/step
rock invoked
1/1 [==============================] - 0s 46ms/step
live long invoked
1/1 [==============================] - 0s 48ms/step
live long invoked
1/1 [==============================] - 0s 39ms/step
fist invoked
1/1 [==============================] - 0s 39ms/step
thumbs up invoked
1/1 [==============================] - 0s 36ms/step
thumbs up invoked
1/1 [==============================] - 0s 44ms/step
thumbs up invoked
1/1 [==============================] - 0s 60ms/step
thumbs up invoked
1/1 [==============================] - 0s 37ms/step
thumbs 

AttributeError: 'cv2.VideoCapture' object has no attribute 'destroyAllWindows'